cài đặt pip

In [ ]:
!pip install flask-cors

check version

In [ ]:
!pip freeze | grep flask_cors

open drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ls

In [ ]:
!ls '/content/drive/MyDrive/ColabNotebooks/DF_test/'

app.ipynb  fruit_detection_model.h5  haarcascade_frontalface_default.xml  index.html  temp


host 5000. step 1 run from eval_js

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://6n13ll7qz7-496ff2e9c6d22116-5000-colab.googleusercontent.com/


code. step 2 run code

In [9]:
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import os
import cv2
from google.colab.output import eval_js

# Khởi tạo Flask app
app = Flask(__name__, template_folder='/content/drive/MyDrive/ColabNotebooks/DF_test/')
CORS(app)

# Load model
model = load_model('/content/drive/MyDrive/ColabNotebooks/DF_test/fruit_detection_model.h5')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Load face cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Tạo thư mục temp nếu chưa tồn tại
if not os.path.exists('temp'):
    os.makedirs('temp')

# Route cho trang chủ
@app.route('/')
def home():
    return render_template('index.html')

# Route cho API predict
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file:
        try:
            # Lưu file tạm thời
            file_path = os.path.join('temp', file.filename)
            file.save(file_path)

            # Đọc và xử lý ảnh
            img = cv2.imread(file_path)
            if img is None:
                return jsonify({'error': 'Invalid image file'}), 400

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            if len(faces) == 0:
                os.remove(file_path)
                return jsonify({'result': 'Not a human face'}), 200

            # Xử lý ảnh cho model
            img = image.load_img(file_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array /= 255.0

            # Dự đoán
            prediction = model.predict(img_array)[0][0]

            # Xóa file tạm
            os.remove(file_path)

            # Xử lý kết quả
            result = 'Fake' if prediction > 0.5 else 'Real'
            confidence = float(prediction) if result == 'Fake' else float(1 - prediction)

            return jsonify({
                'result': result,
                'confidence': round(confidence * 100, 2)  # Làm tròn đến 2 chữ số thập phân
            }), 200

        except Exception as e:
            # Đảm bảo xóa file tạm nếu có lỗi
            if os.path.exists(file_path):
                os.remove(file_path)
            return jsonify({'error': str(e)}), 500

def get_colab_url():
    try:
        return eval_js("google.colab.kernel.proxyPort(5000)")
    except Exception as e:
        print(f"Error getting Colab URL: {e}")
        return None

if __name__ == '__main__':
    # Lấy URL Colab
    COLAB_URL = get_colab_url()
    if COLAB_URL:
        print(f"Server running at: {COLAB_URL}")

    # Chạy app
    #app.run(host='0.0.0.0', port=5000, debug=True)
    app.run()

OSError: Unable to synchronously open file (bad object header version number)